# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

### THE DATA

** Read in the census_data.csv data with pandas**

In [1]:
from google.colab import files
files.upload()
!ls

Saving census_data.csv to census_data.csv
census_data.csv  sample_data


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

In [2]:
dataset = pd.read_csv('census_data.csv')
dataset.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
dataset.describe()

,age,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,10.080679,1077.648844,87.303830,40.437456
std,13.640433,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [4]:
dataset['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [0]:
def label_fix(label):
  if label == ' <=50K':
    return 0
  else:
    return 1

In [0]:
dataset['income_bracket'] = dataset['income_bracket'].apply(label_fix)

In [7]:
dataset.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


### Perform a Train Test Split on the Data

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_data = dataset.drop(['income_bracket'], axis = 1)
y_data = dataset['income_bracket']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=101)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [11]:
dataset.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [12]:
categorical_feats_indices = ['workclass', 'education', 'marital_status', 'occupation',
                             'relationship', 'race', 'gender', 'native_country']

for cate in categorical_feats_indices:
  print(cate)
  print("size: " + str(dataset[cate].unique().size))
  print(dataset[cate].unique())
  print('\n')

workclass
size: 9
[' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked']


education
size: 16
[' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th']


marital_status
size: 7
[' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed']


occupation
size: 15
[' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Craft-repair' ' Transport-moving'
 ' Farming-fishing' ' Machine-op-inspct' ' Tech-support' ' ?'
 ' Protective-serv' ' Armed-Forces' ' Priv-house-serv']


relationship
size: 6
[' Not-in-family' ' Husband' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative']


race
size: 5
[' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other']


gender
size: 2
[' Male' 

In [0]:
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass', hash_bucket_size= 9)
education = tf.feature_column.categorical_column_with_hash_bucket('education', hash_bucket_size= 16)
marital_status = tf.feature_column.categorical_column_with_hash_bucket('marital_status', hash_bucket_size= 7)
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size= 15)
relationship = tf.feature_column.categorical_column_with_hash_bucket('relationship', hash_bucket_size= 6)
race = tf.feature_column.categorical_column_with_hash_bucket('race', hash_bucket_size= 5)
gender = tf.feature_column.categorical_column_with_hash_bucket('gender', hash_bucket_size= 2)
native_country = tf.feature_column.categorical_column_with_hash_bucket('native_country', hash_bucket_size= 42)

** Create the continuous feature_columns for the continuous values using numeric_column **

In [0]:
age = tf.feature_column.numeric_column('age')
education_num	= tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_per_week = tf.feature_column.numeric_column('hours_per_week')

** Put all these variables into a single list with the variable name feat_cols **

In [15]:
X_data.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country'],
      dtype='object')

In [0]:
feat_cols = [age, workclass, education, education_num, marital_status,
             occupation, relationship, race, gender, capital_gain,
             capital_loss, hours_per_week, native_country]

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [0]:
input_func = tf.estimator.inputs.pandas_input_fn(X_train, y_train, 
                                                 batch_size = 100, 
                                                 num_epochs = None, 
                                                 shuffle = True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [18]:
classifier = tf.estimator.LinearClassifier(feature_columns= feat_cols)

W0717 07:39:54.793873 140545815005056 estimator.py:1811] Using temporary folder as model directory: /tmp/tmp6luhq0i0


** Train your model on the data, for at least 5000 steps. **

In [19]:
classifier.train(input_fn= input_func, steps = 5000)

W0717 07:39:54.831330 140545815005056 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0717 07:39:54.862531 140545815005056 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_queue_runner.py:62: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W0717 07:39:54.865655 140545815005056 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_functions.py:500: add_

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [0]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(X_test,
                                                      batch_size = len(X_test), 
                                                      shuffle = False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [0]:
pred_gen = classifier.predict(input_fn= pred_input_func)

** Each item in your list will look like this: **

In [22]:
predictions = list(pred_gen)

W0717 07:40:27.079208 140545815005056 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [0]:
y_pred = []

In [0]:
for pred in predictions:
  y_pred.append(pred['class_ids'][0])

** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [0]:
from sklearn.metrics import classification_report

In [0]:
result = classification_report(y_test, y_pred)

In [27]:
print(result)

              precision    recall  f1-score   support

           0       0.87      0.92      0.89      7436
           1       0.69      0.56      0.62      2333

    accuracy                           0.83      9769
   macro avg       0.78      0.74      0.76      9769
weighted avg       0.83      0.83      0.83      9769



# Great Job!